In [18]:
### FEATURE SELECTION ###

import math
import geopy.distance
from dis import dis
import math
import geopandas as gpd
import numpy
from shapely import wkt
from shapely import wkb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Point
from geopy.geocoders import Nominatim
from os import path
from scipy.spatial import cKDTree
from shapely.geometry import Point

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso


In [25]:

train = pd.read_csv('feature_data/training_set_dropped_no_id.csv')

train = train.drop(['mall_name', 'chain_name', 'lv2', 'lv3', 'lv4', 'municipality_rev_group'], axis=1)

x_train, x_test, y_train, y_test = train_test_split(train.drop('revenue', axis=1), train['revenue'], random_state=104, train_size=0.8, shuffle=True)
features = train.drop('revenue', axis=1).columns.values.tolist()

train.head()
print(features)


['total_nbr_people', 'nbr_people_per_store_in_grunnkrets', 'busstops_within_1500m', 'busstops_within_3000m', 'busstops_within_6000m', 'num_stores_within_500m_and_same_lvl2', 'num_stores_within_20km_and_same_lvl2', 'closest_competitor_lv2', 'num_stores_within_20km_and_same_lvl3', 'closest_competitor_lv3', 'num_stores_within_100m_and_same_lvl4', 'num_stores_within_500m_and_same_lvl4', 'num_stores_within_1km_and_same_lvl4', 'num_stores_within_5km_and_same_lvl4', 'num_stores_within_10km_and_same_lvl4', 'num_stores_within_20km_and_same_lvl4', 'closest_competitor_lv4', 'mean_revenue_for_municipality_rev_group', 'median_revenue_for_municipality_rev_group', 'mean_revenue_lv2', 'mean_revenue_lv3', 'median_revenue_lv2', 'median_revenue_lv3', 'mean_revenue_chain', 'median_revenue_lv4', 'mean_revenue_lv4']


In [26]:
pipeline = Pipeline([
    ('scaler',StandardScaler()),
    ('model',Lasso())
])

In [27]:
search = GridSearchCV(pipeline,
    {'model__alpha':np.arange(0.1,10,0.1)},
    cv = 5, scoring="neg_mean_squared_error",verbose=3
)

In [28]:
search.fit(x_train,y_train)

Fitting 5 folds for each of 99 candidates, totalling 495 fits
[CV 1/5] END .................model__alpha=0.1;, score=-0.537 total time=   0.2s
[CV 2/5] END .................model__alpha=0.1;, score=-0.498 total time=   0.2s
[CV 3/5] END .................model__alpha=0.1;, score=-0.493 total time=   0.1s
[CV 4/5] END .................model__alpha=0.1;, score=-0.495 total time=   0.1s
[CV 5/5] END .................model__alpha=0.1;, score=-0.497 total time=   0.1s
[CV 1/5] END .................model__alpha=0.2;, score=-0.575 total time=   0.1s
[CV 2/5] END .................model__alpha=0.2;, score=-0.545 total time=   0.1s
[CV 3/5] END .................model__alpha=0.2;, score=-0.530 total time=   0.1s
[CV 4/5] END .................model__alpha=0.2;, score=-0.533 total time=   0.1s
[CV 5/5] END .................model__alpha=0.2;, score=-0.534 total time=   0.1s
[CV 1/5] END .model__alpha=0.30000000000000004;, score=-0.634 total time=   0.1s
[CV 2/5] END .model__alpha=0.30000000000000004;

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', Lasso())]),
             param_grid={'model__alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2,
       5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.2, 6.3, 6.4, 6.5,
       6.6, 6.7, 6.8, 6.9, 7. , 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8,
       7.9, 8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9. , 9.1,
       9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9])},
             scoring='neg_mean_squared_error', verbose=3)

In [29]:
search.best_params_

{'model__alpha': 0.1}

In [30]:
coefficients = search.best_estimator_.named_steps['model'].coef_
importance = np.abs(coefficients)
np.array(features)[importance > 0]

array(['busstops_within_1500m', 'num_stores_within_500m_and_same_lvl2',
       'mean_revenue_chain', 'median_revenue_lv4'], dtype='<U41')

In [31]:
np.array(features)[importance == 0]

array(['total_nbr_people', 'nbr_people_per_store_in_grunnkrets',
       'busstops_within_3000m', 'busstops_within_6000m',
       'num_stores_within_20km_and_same_lvl2', 'closest_competitor_lv2',
       'num_stores_within_20km_and_same_lvl3', 'closest_competitor_lv3',
       'num_stores_within_100m_and_same_lvl4',
       'num_stores_within_500m_and_same_lvl4',
       'num_stores_within_1km_and_same_lvl4',
       'num_stores_within_5km_and_same_lvl4',
       'num_stores_within_10km_and_same_lvl4',
       'num_stores_within_20km_and_same_lvl4', 'closest_competitor_lv4',
       'mean_revenue_for_municipality_rev_group',
       'median_revenue_for_municipality_rev_group', 'mean_revenue_lv2',
       'mean_revenue_lv3', 'median_revenue_lv2', 'median_revenue_lv3',
       'mean_revenue_lv4'], dtype='<U41')